<a href="https://colab.research.google.com/github/stepanbabayan/DFBS-Object-Classification/blob/colab/train_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Clone Repository

In [ ]:
!git clone https://github.com/stepanbabayan/DFBS-Object-Classification.git

## Switch to colab branch 

In [ ]:
mv ./DFBS-Object-Classification/ ./Model/

In [ ]:
cd Model

In [ ]:
!git checkout colab

## Additional Environment Setups

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import zipfile
with zipfile.ZipFile('./data.zip', 'r') as zip_ref:
    zip_ref.extractall('')

In [ ]:
import sys
sys.path.append('Model/')

## Imports

In [ ]:
import os
import shutil
import argparse
import yaml

import load_data, models, train
from Model.test import evaluate
from _helpers import make_directory

import torch
from torchsummary import summary

## Environment variables

In [ ]:
use_gpu = True

In [ ]:
# Training Device
if use_gpu:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print('Device: GPU')
else:
    device = torch.device('cpu')
    print('Device: CPU')

## Data path

In [ ]:
# Choose the dataset
num_classes = 5 # Choices: {5, 6, 10}

In [ ]:
classes_5 = ['C-H', 'C-N', 'Mrk SB', 'sdA', 'sdB']
classes_6 = ['C-H', 'C-N', 'Mrk Abs', 'Mrk SB', 'sdA', 'sdB']
classes_10 = ['C Ba', 'C-H', 'C-N', 'C-R', 'Mrk Abs', 'Mrk AGN', 'Mrk SB', 'sdA', 'sdB', 'sdO']

assert num_classes in {5, 6, 10}

if num_classes == 10:
    classes = classes_10
    checkpoint_name = 'Dense_10_Focal_25_3_Final/139.pth'
elif num_classes == 5:
    classes = classes_5
    checkpoint_name = 'Dense_5_High_Focal_25_3_Final/59.pth'
else:
    classes = classes_6
    checkpoint_name = 'Dense_6_High_Focal_25_3_Final/136.pth'

In [ ]:
# Datasets
data_root = f'./data'

if num_classes != 10:
    new_data_root = f'./data_{num_classes}'
    domains = ['test', 'train']

    for domain in domains:
        make_directory(f'{new_data_root}/{domain}')

        for class_name in classes:
            # make_directory(f'{new_data_root}/{domain}/{class_name}')
            shutil.copytree(f'{data_root}/{domain}/{class_name}/', f'{new_data_root}/{domain}/{class_name}/')

    data_root = new_data_root

In [ ]:
train_dir = os.path.join(data_root, 'train')
val_dir = None
test_dir = os.path.join(data_root, 'test')

In [ ]:
print('Num classes:', len(os.listdir(train_dir)))

## Project Parameters

In [ ]:
root_dir = os.path.abspath('./')

In [ ]:
# Train Id is the unique name of the current training process.
# It should be a descriptive name for the current training setup.
train_id = 'Default_10_Notebook_Train_1'

In [ ]:
# The model's final checkpoint will be saved in {model_dir}
model_dir = f'{root_dir}/model/{train_id}'
save_model_path = f"{model_dir}/final.pth"

# All other checkpoints are saved in {checkpoints_dir}
checkpoints_dir = f'{root_dir}/Checkpoint/{train_id}'

if not os.path.exists(model_dir):
    os.makedirs(model_dir)
if not os.path.exists(checkpoints_dir):
    os.makedirs(checkpoints_dir)

## Training Parameters

In [ ]:
# Network parameters
num_epochs = 301
num_classes = len(os.listdir(train_dir))
input_shape = (160, 50)

In [ ]:
# Batch sizes
train_batch_size = 512
val_batch_size = 1
test_batch_size = 256

#### Start the training from scratch

In [ ]:
start_epoch = 0
load_model_path = None

#### In case if you want to continue your training from a certain checkpoint

In [ ]:
# start_epoch = 6
# load_model_path = f'{root_dir}/Checkpoint/{checkpoint_name}'

## Data Loaders

In [ ]:
train_data, train_classes, train_proportions = load_data.load_images(train_dir, train_batch_size, 'train')
val_data, val_classes, _ = load_data.load_images(val_dir, val_batch_size, 'val', _drop_last=False) if val_dir else (None, None, None)
test_data, test_classes, _ = load_data.load_images(test_dir, test_batch_size, 'test', _drop_last=False) if test_dir else (None, None, None)

## Training Setup

In [ ]:
# Model choices: arch = any(['default', 'default_prev', 'default_bn', 'mobilenet', 'resnet'])
#   default: the proposed network
#   default_bn: similar to the proposed, but with more BatchNorm layers
#   default_prev: the network proposed in the previous work
#   mobilenet: MobileNetV2
#   resnet: Resnet

net = models.Model(num_classes=num_classes, input_shape=input_shape, arch='default').to(device)

### Layers

In [ ]:
print(net)

### Output Summary

In [ ]:
summary(net, (1, 160, 50))

In [ ]:
if load_model_path:
    net.load_state_dict(torch.load(load_model_path))

In [ ]:
# If it throws 403 or other error, try switching your browser or turning off incognito mode.
%tensorboard --logdir runs

In [ ]:
print('\nTraining started:')

net = train.train_model(
    net,
    train=train_data,
    val=val_data,
    test=test_data,
    epochs=num_epochs,
    start_epoch=start_epoch,
    device=device,
    model_folder=checkpoints_dir,
    train_id=train_id,
    classes=test_classes,
    train_proportions=train_proportions,
    evaluate=evaluate
)

In [ ]:
torch.save(net.state_dict(), save_model_path)

In [ ]:
# %tensorboard --logdir runs